In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import json

model_path = '/share/huggingface/Meta-Llama-3-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16)

glm生成辅助数据

In [1]:
# TODO:暂未考虑主体同义替换
n = 10
system_prompt = f"""You are an expert in sentence rewriting. Your task is to perform a synonymous rewriting of the input sentence and output {n} rewritten sentences.

Requirements:
1. The semantic meaning of the rewritten sentences must strictly match the original sentence.
2. Use different grammatical structures and expressions to rephrase the input sentence as much as possible.
3. Output format requirements: separate the {n} rewritten sentences with a newline (\n)."""

In [2]:
import json
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="1239bfa4ba1cc7135d4476e3ad30af96.JjzOdCR7qNwFSZk7")

In [ ]:
data_path = './benchmark_ZsRE_ZsRE-test-all.json'
with open(data_path, 'r') as f:
    data_list = json.load(f)

with open('ZsRE-test-all-generate-10.json', 'a') as f:
    for data in data_list[1185:]:
        user_prompt = data['prompt']
        response = client.chat.completions.create(
            model="glm-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
        )
        data['generate_prompt'] = response.choices[0].message.content.split('\n')
        f.write(json.dumps(data, ensure_ascii=False) + '\n')
        f.flush()

In [3]:
data_path = './benchmark_wiki_counterfact_test_cf.json'
with open(data_path, 'r') as f:
    data_list = json.load(f)

with open('wiki_counterfact_test_cf-generate-10.json', 'a') as f:
    for data in data_list[601:]:
        user_prompt = data['prompt']
        response = client.chat.completions.create(
            model="glm-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
        )
        data['generate_prompt'] = response.choices[0].message.content.split('\n')
        f.write(json.dumps(data, ensure_ascii=False) + '\n')
        f.flush()

In [ ]:
user_prompt

测试llm输出稳定

In [ ]:
import transformers
import torch
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [ ]:
# model_id = "meta-llama/Meta-Llama-3-8B"
model_id = "/share/huggingface/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline("text-generation", 
                                 model=model_id, 
                                 model_kwargs={"torch_dtype": torch.bfloat16}, 
                                 device_map="auto")
pipeline("The name of the country of citizenship of Leonardo DiCaprio is")

In [ ]:
pipeline('Which family does Epaspidoceras belong to?')

In [ ]:
model_id = "/share/huggingface/gpt-j-6b"

pipeline = transformers.pipeline("text-generation", 
                                 model=model_id, 
                                 model_kwargs={"torch_dtype": torch.bfloat16}, 
                                 device_map="auto")
pipeline("The name of the country of citizenship of Leonardo DiCaprio is")

In [ ]:
pipeline('Which family does Epaspidoceras belong to?')

In [ ]:
! pip install ipywidgets

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import json

model_path = '/share/huggingface/gpt-j-6b'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16)
prompt = "The name of the country of citizenship of Leonardo DiCaprio is"

pipleline_model = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

pipleline_model(prompt, max_length=100, num_return_sequences=1)